In [82]:
import importlib
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from datetime import datetime
from datetime import date
from datetime import timedelta

In [83]:
import PrjtCF_module as cf

# Outline
* 주거시설 등 개발과 동시에 분양을 하는 사업의 재무모델
* PF대출을 통해 토지비 및 공사비 등 개발사업 자금을 조달
* 금융비용은 PF대출금으로 충당
* 분양대금의 일부는 사업비로, 일부는 대출금 상환재원으로 사용
* PF대출금은 필요에 따라 한도대로 인출

# Input Index Data

In [106]:
# 기간 Index 설정
# prjt index : 사업기간 전 기간에 대한 index로 모델의 base가 되는 index
# cstrn index : 공사기간에 대한 index로 착공일로부터 준공일까지의 기간에 대한 index
# loan index : 대출기간에 대한 index로 대출 실행일로부터 대출 상환일 까지의 기간에 대한 index
idx = cf.PrjtIndex(idxname=['prjt', 'cstrn', 'loan', 'sales'],
                   start=['2021-08', '2021-10', '2021-10', '2021-12'],
                   periods=[24+1, 18+1, 20+1, 16+1],
                   freq='M')

# 공정률 index 설정 : cstrn index 기간 중 적용되는 공정률을 설정
idx.prcs = Series(np.ones(len(idx.cstrn)) / len(idx.cstrn),
                  index=idx.cstrn.index)

# 분양수입금 입금 스케줄 설정
idx.slsidx = idx.sales[[0, 10, 16]]
idx.slsscdd = Series([0.1, 0.4, 0.5], index=idx.slsidx)

# Input Financing Condition Data

In [107]:
equity = cf.Loan(idx, idx, amt_ntnl=10_000)

tra = cf.Loan(idx, idx.loan, amt_ntnl=100_000,
              rate_fee = 0.01, rate_IR = 0.10)

# Input Sales Data

In [108]:
dct_sales = {}

# 분양상품A: 주기에 맞춰 분양수입금 지급
slsA = cf.Account(idx)
slsA.sls_amt = 150_000
slsA.sls_prptn = np.array([0.2, 0.3, 0.2, 0.3]) # sales proportion
slsA.sls_idx = np.array([idx[2], idx[5], idx[9], idx[15]])
slsA.sls_plan = Series(slsA.sls_amt * slsA.sls_prptn,
                       index = slsA.sls_idx)

# Input Cost Data

In [109]:
dct_cost = {}

# 토지비: 최초 1회 지급
lnd = cf.Account(idx)
lnd.addscdd(idx.cstrn[0], 30_000)
dct_cost['lnd'] = lnd

# 공사비: 공정률에 따라 지급
cstrn = cf.Account(idx)
cstrn.addscdd(idx.cstrn.index, 50_000 * idx.prcs)
dct_cost['cstrn'] = cstrn

cost = cf.Merge(dct_cost)
cost.lnd = cost.dct['lnd']
cost.cstrn = cost.dct['cstrn']

# Execution Cash Flow
### 1) 사전 설정

In [110]:
# Make accounts
oprtg = cf.Account(idx) # 운영현금흐름의 입출을 위한 운영계좌
sales = cf.Account(idx) # 분양수입금의 입출을 위한 분양수익금계좌
rpyacc = cf.Account(idx) # 상환자금 관리를 위한 대출금 상환계좌

In [111]:
# Calculate cash amount required and withdraw loan.
class wtdrw_mngmnt:
    def __init__(self, idxno, cstmng):
        self.idxno = idxno
        self.cstmng = cstmng
        
        self.amt_inflw = 0
        self.rsdl_inflw = self.inflw_exptd # 인출필요금액(비용 등)
        
    def inflw_equity(self, eqty):
        """equity instance에 대하여 idxno에 대응하는 인출예정금액(sub_scdd)을
        조회하여 운영계좌로 이체"""
        if eqty.is_wtdrbl:
            amt_inflw = eqty.ntnl.sub_scdd[idxno]
            eqty.ntnl.send(idxno, amt_inflw, oprtg)
        
    def inflw_loan(self, loan):
        """loan instance에 대하여 idxno에 대응하는 누적인출가능잔액 확인,
        누적인출가능잔액 내에서 인출필요금액(비용 등)을 운영계좌로 이체"""
        if loan.is_wtdrbl:
            ntnl_sub_rsdl = loan.ntnl.sub_rsdl_cum[idxno] # 누적인출가능잔액
            tmp_inflw = min(ntnl_sub_rsdl, self.rsdl_inflw)
            # 누적인출가능잔액과 인출필요금액을 비교하여 적은 금액을 대입
        
            loan.ntnl.send(idxno, tmp_inflw, oprtg) 
            # 추가 인출필요금액을 운영계좌로 이체(누적인출가능잔액 내에서)
        
            self.amt_inflw += tmp_inflw # 인출된 금액
            self.rsdl_inflw -= tmp_inflw # 인출 후 잔여 인출필요금액
            
    @property
    def inflw_exptd(self):
        """총 지출필요금액을 확인한 후 운영계좌 잔액을 초과하는 금액
        (추가 인출이 필요한 금액)을 계산하여 반환"""
        cst_exptd = self.cstmng.ttl_exptd # 총 지출필요금액
        oprtg_bal = oprtg.bal_end[idxno] # 운영계좌 잔액
        
        amt_rqrd = max(cst_exptd - oprtg_bal, 0)
        # 지출필요금액에 대하여 운영계좌 잔액으로는 부족한 금액 계산
        return amt_rqrd

In [116]:
# receive sales amount
class sls_mngmnt:
    def __init__(self, idxno):
        self.idxno = idxno
        
        # 해당 인덱스 기간 중 sales amount 입금이 예정되어 있는 금액
        self.slsA_amt = None
    
    # Check sales plan and input sales data
    def make_sls_plan(self):
        try:
            # check sales plan
            sls_amt = slsA.sls_plan.loc[self.idxno]
            try:
                # input sales amount on this index no.
                slsA.addamt(self.idxno, sls_amt)
                # input sales cash schedule on sales cash index.
                slsA.subscdd(idx.slsidx, sls_amt * idx.slsscdd)
            except AttributeError as err:
                print("AttributeError:", err)
        except:
            pass
        
        self.slsA_amt = slsA.sub_rsdl_cum[self.idxno]
    
    # Receive sales amount on sales account
    def rcv_slsamt(self):
        slsA.send(self.idxno, self.slsA_amt, sales)
        
    # Transfer amount to operating account
    def trsf_oprtg(self):
        sales.send(self.idxno, self.slsA_amt, oprtg)

In [117]:
# Calculate expected cost amount and pay cost.
class cst_mngmnt:
    def __init__(self, idxno):
        self.idxno = idxno
        
        # 해당 인덱스 기간 중 cost계좌 상 지출이 예정되어 있는 금액
        self.lnd = cost['lnd'].add_scdd[idxno]
        self.cstrn = cost['cstrn'].add_scdd[idxno]
        self.cstamt = cost.add_scdd[idxno] # cost계좌 상 예정된 지출금의 합계액
        
        # 해당 인덱스 기간 중 loan계좌 상 지출 필요한 수수료 금액
        self.trafee = tra.fee.add_scdd[idxno]
        
        # 해당 인덱스 기간 중 loan계좌 상 지출 필요한 이자 금액
        self.traIR = -tra.ntnl.bal_strt[idxno] * tra.IR.rate
        
    @property
    def ttl_exptd(self):
        """전체 운영비용(cost) 및 금융비용(fee, IR)의 합을 반환"""
        self.ttlsum = self.cstamt
        self.ttlsum = (self.ttlsum +
                       self.trafee + 
                       self.traIR)
        return self.ttlsum
    
    def pay_oprtcst(self):
        """운영계좌에서 운영비용(cost) 지출"""
        # 운영계좌에서 토지비 지출
        oprtg.send(self.idxno, self.lnd, cost['lnd'])
        
        # 운영계좌에서 공사비 지출
        oprtg.send(self.idxno, self.cstrn, cost['cstrn'])
        
    def pay_fnclcst(self):
        """운영계좌에서 금융비용 지출"""
        # 운영계좌에서 대출금 수수료 지급
        oprtg.send(idxno, self.trafee, tra.fee)
        
        # 운영계좌에서 대출금 이자 지급
        oprtg.send(idxno, self.traIR, tra.IR)

In [118]:
# Calculate expected repayment of loan and repay loan.
class repay_mngmnt:
    def __init__(self, idxno, loan):
        self.idxno = idxno
        self.loan = loan
        
        # 상환요구금액 계산
        self.exptd_add_cum = self.loan.ntnl.add_rsdl_cum[self.idxno]
        self.ntnl_bal_end = -self.loan.ntnl.bal_end[self.idxno]
        self.rpy_amt = min(self.exptd_add_cum, self.ntnl_bal_end)
    
    # Transfer repayment amount to repayment account
    def trsf_rpy(self):
        oprtg.send(idxno, self.rpy_amt, rpyacc)
        
    # Repay loan from repayment account
    def rpy_ntnl(self):
        rpyacc.send(idxno, self.rpy_amt, self.loan.ntnl)

### 2) Cash Flow 실행

In [119]:
# Execute cash flow
for idxno in idx.index:
    # If it's initial date then set loan withdrawble.
    equity.set_wtdrbl_intldate(idxno)
    tra.set_wtdrbl_intldate(idxno)
    
    # 해당 인덱스 기간 중 적용할 instance class 생성
    cst = cst_mngmnt(idxno)
    wtdrw = wtdrw_mngmnt(idxno, cst)
    
    # 분양수입대금 sales계좌로 입금 후 운영계좌로 이체
    sls = sls_mngmnt(idxno)
    sls.make_sls_plan()
    sls.rcv_slsamt()
    sls.trsf_oprtg()
    
    # 필요지출금액을 인자로 받아, 조달금액 계산 후 운영계좌 입금
    wtdrw.inflw_equity(equity)
    wtdrw.inflw_loan(tra)
    
    # 운영계좌에서 토지비, 공사비 등 각종 비용 지출
    cst.pay_oprtcst()
    
    # 운영계좌에서 각종 금융비용 지출
    cst.pay_fnclcst()
    
    # 대출금 상환 프로세스 진행
    repay = repay_mngmnt(idxno, tra) # Loan instance class 생성
    repay.trsf_rpy() # 상환 예정 금액을 상환계좌로 이체
    repay.rpy_ntnl() # 상환계좌에서 순서에 따라 상환금 인출
    
    # If it was maturity date then set back loan unwithdrawble.
    equity.setback_wtdrbl_mtrt(idxno)
    tra.setback_wtdrbl_mtrt(idxno)

# Print Result

In [122]:
slsA.df

,add_scdd,sub_scdd,bal_strt,amt_add,amt_sub,bal_end
2021-08-31,0.0,0.0,0.0,0.0,0.0,0.0
2021-09-30,0.0,0.0,0.0,0.0,0.0,0.0
2021-10-31,0.0,0.0,0.0,60000.0,0.0,60000.0
2021-11-30,0.0,0.0,60000.0,0.0,0.0,60000.0
2021-12-31,0.0,30000.0,60000.0,0.0,18000.0,42000.0
2022-01-31,0.0,0.0,42000.0,90000.0,4500.0,127500.0
2022-02-28,0.0,0.0,127500.0,0.0,0.0,127500.0
2022-03-31,0.0,0.0,127500.0,0.0,0.0,127500.0
2022-04-30,0.0,0.0,127500.0,0.0,0.0,127500.0
2022-05-31,0.0,0.0,127500.0,60000.0,3000.0,184500.0


In [121]:
sales.df

,add_scdd,sub_scdd,bal_strt,amt_add,amt_sub,bal_end
2021-08-31,0.0,0.0,0.0,0.0,0.0,0.0
2021-09-30,0.0,0.0,0.0,0.0,0.0,0.0
2021-10-31,0.0,0.0,0.0,0.0,0.0,0.0
2021-11-30,0.0,0.0,0.0,0.0,0.0,0.0
2021-12-31,0.0,0.0,0.0,18000.0,18000.0,0.0
2022-01-31,0.0,0.0,0.0,4500.0,4500.0,0.0
2022-02-28,0.0,0.0,0.0,0.0,0.0,0.0
2022-03-31,0.0,0.0,0.0,0.0,0.0,0.0
2022-04-30,0.0,0.0,0.0,0.0,0.0,0.0
2022-05-31,0.0,0.0,0.0,3000.0,3000.0,0.0
